In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
df_dates = pd.DataFrame({
    'week_start_date': pd.date_range(start='1980-01-01', end='2018-01-01', freq='W-MON').values
})
df_dates['year'] = df_dates['week_start_date'].apply(lambda x: int(x.strftime("%Y")))
df_dates['weekofyear'] = df_dates['week_start_date'].apply(lambda x: int(x.strftime("%U")))
df_dates.head(n=2)

In [ ]:
df_preds = []
for fn_prev in [
    '1-submission_20180530_092740-score_29.csv', # copy last year
    ##'2.0-submission_20180530_111647-score_29.csv', # GLM for both cities
    '2.1-submission_20180530_133617-score_26.csv', # GLM per city
    #'3.0-submission_20180530_141052-score_26.csv', # RF basic
    '3.0.1-submission_20180531_062302.csv', # RF with refit
    # '3.1-submission_20180530_165638.csv', # RF with diff
    # '3.1B-submission_20180531_062845.csv', # RF with diff and ...?
    # '3.1.1-submission_20180531_104829.csv', # RF with target being diff
    # '3.2-submission_20180531_095313.csv', # RF with diff and classifier
    #'3.2-submission_20180531_134029-score_27.csv', # RF with classifier and regressor
    '4.0-submission_20180531_083801-score_22.csv', # LSTM regression: basic
    #'4.0.1A-submission_20180607_072148.csv', # LSTM regression with deseasoning and all features
    #'4.0.1B-submission_20180607_082815.csv', # LSTM regression with deseasoning and using subset of features
    #'4.0.1C-submission_20180607_090241.csv', # LSTM regression with deseason and subset from 3.0.1 and lookback=60
    # '4.0.1D-submission_20180607_100818.csv', # ditto with all features
    '4.0.1E-submission_20180607_102939.csv', # ditto with lstm(100)
    #'5-submission_20180605_111301.csv',
    # '5-submission_20180605_113132.csv',
    #'5.1A-submission_20180606_092324-score_30.csv', # lasso + RF, n_back=10
    #'5.1B-submission_20180606_093559-score_29.csv', # lasso + linear, n_back=10
    # '5.1C-submission_20180606_100819-score_38.csv', # lasso + linear, n_back=50
    ]:
    df_prev = pd.read_csv('data/interim/%s'%fn_prev)
    df_prev['fn'] = fn_prev
    df_preds.append(df_prev)
    
df_preds = pd.concat(df_preds, axis=0)
df_preds = df_preds.merge(df_dates, on=['year', 'weekofyear'], how='left')
df_preds = df_preds.pivot_table(index=['city', 'week_start_date'], columns='fn', values='total_cases')
df_preds.head(n=2)

In [ ]:
for city in ['sj','iq']:
    df_preds.loc[city].plot(cmap='Blues', figsize=(20,5))
    plt.title(city)
    plt.legend()
    plt.show()

## plot ensemble prediction for fun

- '1-submission_20180530_092740-score_29.csv', # copy last year
-  '2.1-submission_20180530_133617-score_26.csv', # GLM per city
-    '3.0.1-submission_20180531_062302.csv', # RF with refit
-    '4.0-submission_20180531_083801-score_22.csv', # LSTM regression: basic
-    '4.0.1E-submission_20180607_102939.csv'

got whooping score=23

In [ ]:
for city in ['sj','iq']:
    df_preds.loc[city].mean(axis=1).plot(figsize=(20,5))
    plt.title(city)
    plt.legend()
    plt.show()

In [ ]:
predictions = df_preds.mean(axis=1).reset_index().rename(columns={0: 'total_cases'})
predictions['total_cases'] = predictions['total_cases'].astype('int')
predictions = predictions.merge(df_dates, on=['week_start_date'], how='left')
predictions = predictions.set_index(['city','week_start_date'])

from src.features.build_features import load_raw
df_all = load_raw()
submit = df_all['submission'].copy().reset_index()

submit = submit.merge(predictions, how='left', on=['city','year','weekofyear'], suffixes=['_zero', ''])

del submit['total_cases_zero']
submit = submit.fillna(value=0)
submit['total_cases'] = submit['total_cases'].astype('int')

submit.shape

In [ ]:
submit.describe()

In [ ]:
from src.features.build_features import make_submission
make_submission(submit)